In [1]:
import pandas as pd 
import numpy as np 
import datetime
import sqlite3 
import pybaseball as pyb #Pybaseball is where we will be pulling data from

In [2]:
# Get today's date 
today = datetime.date.today()
#turn the output of the date to a string to be able to pass through the statcast function
today_str = today.strftime('%Y-%m-%d')  
#This start_dt will be the starting date of spring training for the 2023 season 
start_dt_od = '2023-03-30'


In [ ]:
#Get all data leading from Opening Day to the current date from pybaseball's statcast function 
data = pyb.statcast(start_dt=start_dt_od, end_dt=today_str, team=None, verbose=True, parallel=True)

#Using an arbitray set of dates to test to make sure the functions work and data can be written to the databse - this line is only included to test the code out

In [4]:
#data = pyb.statcast(start_dt = '2022-05-17', end_dt = '2022-05-18', team = None, verbose = True, parallel = True) 

In [ ]:
#can use this cell as a test to see what the data looks like
#print(data.tail(1))

#This is a function to add some aditional things I want in the pitch by pich data

In [5]:
def add_to_savant(df):
    # Modify 'pfx_x' and 'pfx_z' columns to get them in inches 
    df['pfx_x'] = df['pfx_x'].apply(lambda x: x * -12)
    df['pfx_z'] = df['pfx_z'].apply(lambda x: x * 12)
    
    # Create 'hard_hit' column
    df['hard_hit'] = 0  # Initialize all values to 0
    mask = df['launch_speed'].notna()  # Get a boolean mask of non-missing values in 'launch_speed'
    df.loc[mask & (df['launch_speed'] > 95), 'hard_hit'] = 1  # Set values to 1 where condition is met
    
    # Create 'sweet_spot' column
    df['sweet_spot'] = 0  # Initialize all values to 0
    mask = df['launch_angle'].notna()  # Get a boolean mask of non-missing values in 'launch_angle'
    df.loc[mask & ((df['launch_angle'] >= 8) & (df['launch_angle'] <= 32)), 'sweet_spot'] = 1  # Set values to 1 where condition is met

    # Add 'VAA' column
    vy_f = -1 * np.sqrt(df['vx0']**2 + df['vy0']**2) 
    t = (vy_f - df['vy0']) / df['ay']
    vz_f = df['vz0'] + df['az'] * t
    df['VAA'] = np.round(-1 * np.arctan(vz_f / vy_f) * (180 / np.pi), 2)
    
    # Add 'Count' column
    df['count'] = df['balls'].astype(str) + '-' + df['strikes'].astype(str)

    # Add 'count_type' column  
    df['count_type'] = ''
    df.loc[df['count'].isin(['1-0', '2-0', '3-0', '2-1', '3-1', '3-2']), 'count_type'] = 'hitter'
    df.loc[df['count'].isin(['0-1', '0-2', '1-2']), 'count_type'] = 'pitcher'
    df.loc[df['count'].isin(['0-0', '1-1', '2-2']), 'count_type'] = 'even'  

    # Add a column, 'swing' to determine if the pitch was swung a or not (1=swing) 
    #this classification includes all bunt attempts as swings so you will need to filter those out in later analysis
    #define the strings to look for
    target_strings = ['hit_into_play', 'foul', 'swinging_strike', 'swinging_strike_blocked', 'foul_bunt', 'foul_tip', 'foul_pitchout', 'missed_bunt', 'bunt_foul_tip', 'swinging_pitchout']
    # create a new column with 1s and 0s based on whether the target strings are present in column1
    df['swing'] = np.where(df['description'].isin(target_strings), 1, 0)  

    # Create a new column 'swing_type'
    swing_types = df['description'].copy()
    
    # Determine if a pitch resulted in: contact, foul, whiff, take_ball, take_strike, or undef (undefined)
    swing_types.loc[swing_types.isin(['hit_into_play'])] = 'contact'
    swing_types.loc[swing_types.isin(['foul', 'foul_bunt', 'foul_tip', 'bunt_foul_tip', 'foul_pitchout'])] = 'foul'
    swing_types.loc[swing_types.isin(['swinging_strike', 'swinging_strike_blocked', 'missed_bunt', 'swinging_pitchout'])] = 'whiff'
    swing_types.loc[swing_types.isin(['ball', 'blocked_ball', 'hit_by_pitch', 'pitchout'])] = 'take_ball'
    swing_types.loc[swing_types.isin(['called_strike'])] = 'take_strike'
    swing_types.loc[~swing_types.isin(['contact', 'foul', 'whiff', 'take_ball', 'take_strike'])] = 'undef'

    # Add the new column to the dataframe
    df['swing_type'] = swing_types 

    return df

#These are the column names I will be changing to make querying quicker for me

In [6]:
# Define a dictionary with the column name changes
column_names_dict = {
    'release_speed': 'velo',
    'release_pos_x': 'release_side',
    'release_pos_z': 'release_height',
    'game_date': 'date',
    'pf_x_z': 'ind_vert_break',
    'pf_x_x': 'horizontal_break',
    'launch_speed': 'exit_velo',
    
    # Add more column names as needed
}

#Function to apply column name changes to the data

In [7]:
def rename_statcast_columns(data, column_names_dict):
    """
    Rename columns from the Statcast data that was pulled from Savant

    Parameters:
        data (pd.DataFrame): Statcast data from Savant
        column_names_dict (dict): a dictionary containing the current column names as keys
                                  and the new column names as values

    Returns:
        pd.DataFrame: the modified DataFrame with the renamed columns
    """
    data = data.rename(columns=column_names_dict)
    return data

In [8]:
# Run the downloaded data from pybaseball for the day through the 'add_to_savant' function
modified_data = add_to_savant(data)

In [ ]:
# Run the new dataframe (modified_data) that contains the new columns from the 'add_to_savant' function through 
# the function that renames the columns
data_cleaned = rename_statcast_columns(modified_data, column_names_dict) 
#print(data_cleaned.head())

#We want to clear out any data in the database table before we load anything new. This is to ensure that if Statcast makes any changes we will always have the most up to date information - thanks to Jeremy Maschino for this suggestion

In [10]:
# Connect to the database
conn = sqlite3.connect('2023Statcast.db')

# Create a cursor object to execute SQL statements
cur = conn.cursor()

# Execute the DROP TABLE statement to remove the entire table
cur.execute('DROP TABLE statcast_data_2023')

# Commit the changes to the database
conn.commit()

# Close the cursor and database connections
cur.close()
conn.close()

#We will now put all of our updated data into the table 'statcast_data_2023' into the '2023Statcast' databas

In [11]:
#Connect to the SQLLite database 2023Statcast
conn = sqlite3.connect('2023Statcast.db')

#define the table name
table_name = 'statcast_data_2023'
#if the table name exists then append the data on it
data_cleaned.to_sql(table_name, conn, if_exists='append', index=False)

# Close the connection
conn.close()
